# Лабораторная работа №7 "Классификация обзоров фильмов"
### Выполнила студентка группы БВТ2101 Пьянова Анна Олеговна

### Цель работы
Классификация последовательностей - это проблема прогнозирующего моделирования,
когда у вас есть некоторая последовательность входных данных в пространстве или
времени, и задача состоит в том, чтобы предсказать категорию для последовательности.
Проблема усложняется тем, что последовательности могут различаться по длине,
состоять из очень большого словарного запаса входных символов и могут потребовать от
модели изучения долгосрочного контекста или зависимостей между символами во входной
последовательности.

В данной лабораторной работе также будет использоваться датасет IMDb, однако
обучение будет проводиться с помощью рекуррентной нейронной сети.

### Задачи:
- Ознакомиться с рекуррентными нейронными сетями
- Изучить способы классификации текста
- Ознакомиться с ансамблированием сетей
- Построить ансамбль сетей, который позволит получать точность не менее 97%


### Выполнение работы

In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Embedding, Dropout, BatchNormalization
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import RMSprop
from keras.datasets import imdb

Загрузка данных

In [2]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=5000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

Обрезка и дополнение входных последовательностей

In [3]:
max_review_length = 500
top_words = 5000
embedding_vecor_length = 32

X_train = data[10000:]
y_train = targets[10000:]
X_test = data[:10000]
y_test = targets[:10000]

X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [17]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 323ms/step - accuracy: 0.7222 - loss: 0.5081 - val_accuracy: 0.8381 - val_loss: 0.3787
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 200s 320ms/step - accuracy: 0.8820 - loss: 0.2960 - val_accuracy: 0.8506 - val_loss: 0.3507
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 153s 245ms/step - accuracy: 0.9003 - loss: 0.2553 - val_accuracy: 0.8840 - val_loss: 0.2871
Accuracy: 88.40%


In [19]:
model.save_weights('first.weights.h5')
model.save('first.keras')

In [20]:
model_lstm = Sequential()
model_lstm.add(Embedding(top_words, embedding_vecor_length))
model_lstm.add(LSTM(100))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1, activation='sigmoid'))

In [21]:
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_lstm.summary())
model_lstm.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model_lstm.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 249ms/step - accuracy: 0.7215 - loss: 0.5111 - val_accuracy: 0.8731 - val_loss: 0.3161
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 150s 241ms/step - accuracy: 0.8651 - loss: 0.3244 - val_accuracy: 0.8793 - val_loss: 0.2950
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 155s 248ms/step - accuracy: 0.9041 - loss: 0.2428 - val_accuracy: 0.8833 - val_loss: 0.2836
Accuracy: 88.33%


In [22]:
model_lstm.save_weights('lstm.weights.h5')
model_lstm.save('lstm.keras')

In [9]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 101ms/step - accuracy: 0.7372 - loss: 0.4817 - val_accuracy: 0.8954 - val_loss: 0.2744
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 96ms/step - accuracy: 0.9209 - loss: 0.2110 - val_accuracy: 0.8928 - val_loss: 0.2980
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 97ms/step - accuracy: 0.9420 - loss: 0.1615 - val_accuracy: 0.8986 - val_loss: 0.2659
Accuracy: 89.86%


In [23]:
model_conv = Sequential()
model_conv.add(Embedding(top_words, embedding_vecor_length))
model_conv.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model_conv.add(MaxPooling1D(pool_size=2))
model_conv.add(Dropout(0.2))
model_conv.add(LSTM(100))
model_conv.add(Dropout(0.2))
model_conv.add(Dense(1, activation='sigmoid'))

In [24]:
model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_conv.summary())
model_conv.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model_conv.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 65s 101ms/step - accuracy: 0.6973 - loss: 0.5239 - val_accuracy: 0.8780 - val_loss: 0.3023
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 70s 111ms/step - accuracy: 0.9037 - loss: 0.2454 - val_accuracy: 0.8914 - val_loss: 0.2671
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 76s 121ms/step - accuracy: 0.9227 - loss: 0.2018 - val_accuracy: 0.8968 - val_loss: 0.2540
Accuracy: 89.68%


In [25]:
model_conv.save_weights('conv.weights.h5')
model_conv.save('conv.keras')

In [10]:
model_batch = Sequential()
model_batch.add(Embedding(top_words, embedding_vecor_length))
model_batch.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model_batch.add(MaxPooling1D(pool_size=2))
model_batch.add(Dropout(0.2))
model_batch.add(LSTM(100))
model_batch.add(BatchNormalization())
model_batch.add(Dropout(0.2))
model_batch.add(Dense(1, activation='sigmoid'))

In [11]:
model_batch.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])
print(model_batch.summary())
model_batch.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model_batch.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 80s 125ms/step - accuracy: 0.7866 - loss: 0.4373 - val_accuracy: 0.7242 - val_loss: 0.6169
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 75s 120ms/step - accuracy: 0.9114 - loss: 0.2203 - val_accuracy: 0.6497 - val_loss: 1.2479
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 76s 122ms/step - accuracy: 0.9330 - loss: 0.1741 - val_accuracy: 0.8382 - val_loss: 0.3902
Accuracy: 83.82%


Ансамблировние моделей

In [4]:
model_lstm = load_model('lstm.keras')
model_lstm.load_weights('lstm.weights.h5')

model_conv = load_model('conv.keras')
model_conv.load_weights('conv.weights.h5')

model = load_model('first.keras')
model.load_weights('first.weights.h5')

weights = [0.3, 0.5, 0.2]

def predict_ensemble(text, weights):
    index = imdb.get_word_index()
    
    text_sequence = [index.get(word, 2) for word in text.lower().split()]
    
    text_sequence = sequence.pad_sequences([text_sequence], maxlen=max_review_length)
    
    pred_lstm = model_lstm.predict(text_sequence, verbose=0)
    pred_conv = model_conv.predict(text_sequence, verbose=0)
    pred = model.predict(text_sequence, verbose=0)
    
    ensemble_pred = np.average([pred_lstm, pred_conv, pred], axis=0, weights=weights)

    sentiment = "positive" if ensemble_pred > 0.5 else "negative"
    confidence = ensemble_pred[0][0] if sentiment == "positive" else 1 - ensemble_pred[0][0]
    
    return sentiment, confidence, text

In [29]:
user_text = input("Your text: ")
sentiment, confidence, text = predict_ensemble(user_text, weights)
print(f"{text}\n Sentiment: {sentiment} ({confidence:.2f})")

This is movie is messed up and I'll give it just one star since maybe at least some messed up people
 Sentiment: negative (0.72)


In [5]:
user_text = input("Your text: ")
sentiment, confidence, text = predict_ensemble(user_text, weights)
print(f"{text}\n Sentiment: {sentiment} ({confidence:.2f})")

this movie is not bad
 Sentiment: positive (0.65)
